# Setup and imports


Install PyTorch, Torchvision, and tqdm into the Colab runtime (suppressing verbose output).

In [ ]:
!pip install torch torchvision tqdm --quiet

Import PyTorch core, neural-net layer (nn), optimzers, the dataloader, TorchVision utilities (datasets, transforms, model zoo), tqdm progress bars, Colab Drive API, Matplotlib for plotting, a function to refresh notebook output, and os for paths.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
from google.colab import drive
import matplotlib.pyplot as plt
from IPython.display import clear_output
import os

Mount your Google Drive at `/content/drive` so reads/writes persist outside the ephemeral VM.



In [ ]:
drive.mount('/content/drive')

Pick GPU if available; otherwise CPU. Print which one you're using.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

### Dataset paths and transforms
Define the folders where your training/validation images live (ImageFolder layout).

In [ ]:
train_dir = '/content/drive/MyDrive/dataset/train'
val_dir   = '/content/drive/MyDrive/dataset/val'

Build the **training** preprocessing/augmentation pipeline:

*   resize to ResNet's $224 \times 224$,
*   random flips/rotations/color jitter for augmentation,
*   convert to tensor,
*   normalize with ImageNet mean/std (to match pretrained weights' expectations).





In [ ]:
train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

Build the **validation** pipeline: deterministic resize + normalization (no augmentation).

In [ ]:
val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

Create dataset objects that read subfolders class labels and apply transforms.

In [ ]:
train_ds = datasets.ImageFolder(train_dir, transform=train_tfms)
val_ds   = datasets.ImageFolder(val_dir, transform=val_tfms)

Count classes from subfolder names and print them.

In [ ]:
num_classes = len(train_ds.classes)
print(f"Detected {num_classes} classes:", train_ds.classes)

Wrap datasets into iterable mini-batches:
*   64 images per batch,
*   shuffle train, keep val deterministic,
* use 3 worker processes for background loading.

In [ ]:
batch_size = 64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)

# Model setup (Transfer Learning)

Load a ResNet-18 preinitialized on ImageNet (strong generic features).

In [ ]:
model = models.resnet18(weights='IMAGENET1K_V1')

**Freeze** all backbone parameters so only the classifier head trains (fast, avoids overfitting).

In [ ]:
for param in model.parameters():
    param.requires_grad = False

Replace the final fully connected layer with a new classifier matching your class count.

In [ ]:
model.fc = nn.Linear(model.fc.in_features, num_classes)

Move model weights to GPU (or CPU) memory.

In [ ]:
model = model.to(device)

Define the multi-class classification loss (negative log-likelihood over softmax).

In [ ]:
criterion = nn.CrossEntropyLoss()

Optimize only the new head's parameters using Adam at 1e-3 learning rate.

In [ ]:
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
# After every 7 epochs, multiply the learnine rate by 0.1 (learning-rate decacy).
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Directory for checkpoints
ckpt_dir = '/content/drive/MyDrive/checkpoints'
os.makedirs(ckpt_dir, exist_ok=True)

# Training loop


Train for 20 epochs and prepare lists to log metrics.

In [ ]:
num_epochs = 20
train_losses, val_losses, val_accuracies = [], [], []

Start training!

In [ ]:
for epoch in range(num_epochs):
    # Switch to training mode (enables dropout/batchnorm updates).
    model.train()

    # Zero the epoch loss accumulator.
    running_loss = 0.0
    # Create a progress bar over training batches.
    loop = tqdm(train_dl, desc=f"Epoch {epoch+1}/{num_epochs}")

    # Iterate over batches.
    for x, y in loop:
        # Move images/labels to GPU.
        x, y = x.to(device), y.to(device)
        # Clear gradients from the previous step.
        optimizer.zero_grad()
        # Forward pass: logits for the batch
        out = model(x)

        # Compute cross-entropy loss against true labels.
        loss = criterion(out, y)
        # Backpropagate: compute gradients w.r.t. trainable parameteres (the head).
        loss.backward()
        # Take an optimizer step: update head weights.
        optimizer.step()

        # Accumulate the sum of batch losses (scaled by batch size) for epoch mean.
        running_loss += loss.item() * x.size(0)
        # Show current batch loss in the tqdm bar.
        loop.set_postfix(loss=loss.item())

    # Compute and store the mean training loss over the full epoch.
    epoch_train_loss = running_loss / len(train_dl.dataset)
    train_losses.append(epoch_train_loss)

    # ----- Validation -----
    # Switch to eval mode (disables dropout; uses running stats).
    model.eval()
    # Init validation accumulators.
    val_loss, correct = 0.0, 0

    # Validation loop
    with torch.no_grad():
        for x, y in val_dl:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            preds = out.argmax(1)
            correct += (preds == y).sum().item()

    # Compute mean validation loss and accuracy; log them.
    epoch_val_loss = val_loss / len(val_dl.dataset)
    epoch_val_acc  = correct / len(val_dl.dataset)
    val_losses.append(epoch_val_loss)
    val_accuracies.append(epoch_val_acc)

    # Update the learning rate per schedule (decays every 7 epochs).
    scheduler.step()

    # ----- Live plot -----
    clear_output(wait=True)
    plt.figure(figsize=(8,4))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title(f"Epoch {epoch+1}/{num_epochs} | Val Acc: {epoch_val_acc:.3f}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {epoch_train_loss:.4f} | "
          f"Val Loss: {epoch_val_loss:.4f} | "
          f"Val Acc: {epoch_val_acc:.4f}")

    # ----- Save checkpoint -----
    ckpt_path = f"{ckpt_dir}/resnet18_epoch_{epoch+1}.pth"
    torch.save(model.state_dict(), ckpt_path)
    print(f"✅ Saved checkpoint: {ckpt_path}")

# Post-training summary plots

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8,4))
plt.plot(val_accuracies, label='Validation Accuracy', color='green')
plt.title("Validation Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

print("🎯 Training complete. Best model stored in:", ckpt_dir)